In [1]:
from typing import List
from pathlib import Path

from unidecode import unidecode
from pyspark.sql import SparkSession, DataFrame, Row
import pyspark.sql.functions as F
from pyspark.sql.types import StringType

In [2]:
JAR_PACKAGES = ",".join([str(x) for x in Path("../../jars").glob("*.jar")])
JAR_PACKAGES

'../../jars/antlr4-runtime-4.9.3.jar,../../jars/aws-java-sdk-bundle-1.12.392.jar,../../jars/delta-core_2.12-2.4.0.jar,../../jars/delta-storage-2.4.0.jar,../../jars/hadoop-aws-3.3.1.jar,../../jars/wildfly-openssl-1.0.7.Final.jar'

## Read The Datasets

## Overview of Each Dataset


### Dataset High Level Overview

For each Dataset get the ff. (tabular)
- Number of Rows
- Number of Columns
- Number of Null Values
- Number Columns with Null Values
- List of Columns with Null

In [3]:
spark = SparkSession.builder.appName("olist_data_profiling").master("spark://spark:7077") \
            .config("spark.jars", JAR_PACKAGES) \
            .config("spark.hadoop.fs.s3a.access.key","datalake") \
            .config("spark.hadoop.fs.s3a.secret.key","datalake") \
            .config("spark.hadoop.fs.s3a.endpoint","http://minio:9000") \
            .config("spark.hadoop.fs.s3a.path.style.access", "true") \
            .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
            .getOrCreate()

In [4]:
bronze_container_path = "s3a://bronze"

In [5]:
customers_df = spark.read.format("csv") \
    .option("path", f"{bronze_container_path}/olist/olist_customers_dataset.csv") \
    .option("header", True) \
    .option("inferSchema", True) \
    .load()
customers_df.printSchema()

root
 |-- customer_id: string (nullable = true)
 |-- customer_unique_id: string (nullable = true)
 |-- customer_zip_code_prefix: integer (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)



In [6]:
geolocation_df = spark.read.format("csv") \
    .option("path", f"{bronze_container_path}/olist/olist_geolocation_dataset.csv") \
    .option("header", True) \
    .option("inferSchema", True) \
    .load()
geolocation_df.printSchema()

root
 |-- geolocation_zip_code_prefix: integer (nullable = true)
 |-- geolocation_lat: double (nullable = true)
 |-- geolocation_lng: double (nullable = true)
 |-- geolocation_city: string (nullable = true)
 |-- geolocation_state: string (nullable = true)



In [7]:
order_items_df = spark.read.format("csv") \
    .option("path", f"{bronze_container_path}/olist/olist_order_items_dataset.csv") \
    .option("header", True) \
    .option("inferSchema", True) \
    .load()
order_items_df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- order_item_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- seller_id: string (nullable = true)
 |-- shipping_limit_date: timestamp (nullable = true)
 |-- price: double (nullable = true)
 |-- freight_value: double (nullable = true)



In [8]:
order_payments_df = spark.read.format("csv") \
    .option("path", f"{bronze_container_path}/olist/olist_order_payments_dataset.csv") \
    .option("header", True) \
    .option("inferSchema", True) \
    .load()
order_payments_df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- payment_sequential: integer (nullable = true)
 |-- payment_type: string (nullable = true)
 |-- payment_installments: integer (nullable = true)
 |-- payment_value: double (nullable = true)



In [9]:
order_reviews_df = spark.read.format("csv") \
    .option("path", f"{bronze_container_path}/olist/olist_order_reviews_dataset.csv") \
    .option("header", True) \
    .option("inferSchema", True) \
    .load()
order_reviews_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- review_score: string (nullable = true)
 |-- review_comment_title: string (nullable = true)
 |-- review_comment_message: string (nullable = true)
 |-- review_creation_date: string (nullable = true)
 |-- review_answer_timestamp: string (nullable = true)



In [10]:
orders_df = spark.read.format("csv") \
    .option("path", f"{bronze_container_path}/olist/olist_orders_dataset.csv") \
    .option("header", True) \
    .option("inferSchema", True) \
    .load()
orders_df.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- order_status: string (nullable = true)
 |-- order_purchase_timestamp: timestamp (nullable = true)
 |-- order_approved_at: timestamp (nullable = true)
 |-- order_delivered_carrier_date: timestamp (nullable = true)
 |-- order_delivered_customer_date: timestamp (nullable = true)
 |-- order_estimated_delivery_date: timestamp (nullable = true)



In [11]:
products_df = spark.read.format("csv") \
    .option("path", f"{bronze_container_path}/olist/olist_products_dataset.csv") \
    .option("header", True) \
    .option("inferSchema", True) \
    .load()
products_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_category_name: string (nullable = true)
 |-- product_name_lenght: integer (nullable = true)
 |-- product_description_lenght: integer (nullable = true)
 |-- product_photos_qty: integer (nullable = true)
 |-- product_weight_g: integer (nullable = true)
 |-- product_length_cm: integer (nullable = true)
 |-- product_height_cm: integer (nullable = true)
 |-- product_width_cm: integer (nullable = true)



In [12]:
sellers_df = spark.read.format("csv") \
    .option("path", f"{bronze_container_path}/olist/olist_sellers_dataset.csv") \
    .option("header", True) \
    .option("inferScema", True) \
    .load()
sellers_df.printSchema()

root
 |-- seller_id: string (nullable = true)
 |-- seller_zip_code_prefix: string (nullable = true)
 |-- seller_city: string (nullable = true)
 |-- seller_state: string (nullable = true)



In [13]:
product_category_name_translation_df = spark.read.format("csv") \
    .option("path", f"{bronze_container_path}/olist/product_category_name_translation.csv") \
    .option("header", True) \
    .option("inferScema", True) \
    .load()
product_category_name_translation_df.printSchema()

root
 |-- product_category_name: string (nullable = true)
 |-- product_category_name_english: string (nullable = true)



### Dataset Detailed Info

Create a tabular data of the ff.

- Column name
- Null amount
- Null percentage among the respective dataset
- Data type
- total categorical entries

In [39]:
def show_detailed_stats(df_list: List[DataFrame]):
    columns = ["dataset_name", "column_name", "row_count", "col_count", "null_amount", "null_percentage", "data_type", "unique_count"]
    all_df = []
    
    for df in df_list:
        data = [
            (
                df.select(F.element_at(F.split(F.input_file_name(), "/"), -1)).first()[0],
                c, 
                df.count(), 
                len(df.columns), 
                df.where(df[c].isNull()).count(), 
                round(100 * df.where(df[c].isNull()).count() / df.count(), 2),
                dict(df.dtypes)[c], 
                df.select(c).distinct().count()
             ) 
                 for c in df.columns
        ]
        all_df.extend(data)
        all_df.append(("-", "-", "-", "-", "-", "-", "-", "-"))
    rdd = spark.sparkContext.parallelize(all_df)
    result_df = rdd.toDF(columns)
    return result_df

stats_df = show_detailed_stats([
    customers_df,
    geolocation_df,
    order_items_df,
    order_payments_df,
    order_reviews_df,
    orders_df,
    products_df,
    sellers_df,
    product_category_name_translation_df,
])
stats_df.show(n=100, truncate=False)

+-------------------------------------+-----------------------------+---------+---------+-----------+---------------+---------+------------+
|dataset_name                         |column_name                  |row_count|col_count|null_amount|null_percentage|data_type|unique_count|
+-------------------------------------+-----------------------------+---------+---------+-----------+---------------+---------+------------+
|olist_customers_dataset.csv          |customer_id                  |99441    |5        |0          |0.0            |string   |99441       |
|olist_customers_dataset.csv          |customer_unique_id           |99441    |5        |0          |0.0            |string   |96096       |
|olist_customers_dataset.csv          |customer_zip_code_prefix     |99441    |5        |0          |0.0            |int      |14994       |
|olist_customers_dataset.csv          |customer_city                |99441    |5        |0          |0.0            |string   |4119        |
|olist_custom

In [112]:
order_id = "8272b63d03f5f79c56e9e4120aec44ef"
order_payments_df.where(F.col("order_id") == order_id).show(n=100, truncate=False)
order_items_df.where(F.col("order_id") == order_id).show(n=100, truncate=False)

+--------------------------------+------------------+------------+--------------------+-------------+
|order_id                        |payment_sequential|payment_type|payment_installments|payment_value|
+--------------------------------+------------------+------------+--------------------+-------------+
|8272b63d03f5f79c56e9e4120aec44ef|1                 |credit_card |2                   |196.11       |
+--------------------------------+------------------+------------+--------------------+-------------+

+--------------------------------+-------------+--------------------------------+--------------------------------+-------------------+-----+-------------+
|order_id                        |order_item_id|product_id                      |seller_id                       |shipping_limit_date|price|freight_value|
+--------------------------------+-------------+--------------------------------+--------------------------------+-------------------+-----+-------------+
|8272b63d03f5f79c56e9e41

In [110]:
order_items_df.groupBy("order_id").agg(F.count("order_id").alias("total_orders")).orderBy(F.desc("total_orders")).show(truncate=False)

+--------------------------------+------------+
|order_id                        |total_orders|
+--------------------------------+------------+
|8272b63d03f5f79c56e9e4120aec44ef|21          |
|1b15974a0141d54e36626dca3fdc731a|20          |
|ab14fdcfbe524636d65ee38360e22ce8|20          |
|428a2f660dc84138d969ccd69a0ab6d5|15          |
|9ef13efd6949e4573a18964dd1bbe7f5|15          |
|73c8ab38f07dc94389065f7eba4f297a|14          |
|9bdc4d4c71aa1de4606060929dee888c|14          |
|37ee401157a3a0b28c9c6d0ed8c3b24b|13          |
|2c2a19b5703863c908512d135aa6accc|12          |
|af822dacd6f5cff7376413c03a388bb7|12          |
|3a213fcdfe7d98be74ea0dc05a8b31ae|12          |
|637617b3ffe9e2f7a2411243829226d0|12          |
|c05d6a79e55da72ca780ce90364abed9|12          |
|71dab1155600756af6de79de92e712e3|11          |
|5a3b1c29a49756e75f1ef513383c0c12|11          |
|7f2c22c54cbae55091a09a9653fd2b8a|11          |
|6c355e2913545fa6f72c40cbca57729e|11          |
|9aec4e1ae90b23c7bf2d2b3bfafbd943|10    

In [22]:
# order_items_df.groupBy("order_id").agg(F.countDistinct("seller_id").alias("cnt")).where("cnt > 3").show(truncate=False)

order_items_df.where("order_id = '8c2b13adf3f377c8f2b06b04321b0925'").show()

+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|            order_id|order_item_id|          product_id|           seller_id|shipping_limit_date| price|freight_value|
+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+
|8c2b13adf3f377c8f...|            1|6f59fe49d85eb1353...|977f9f63dd360c2a3...|2017-11-23 20:31:40|  90.9|        21.08|
|8c2b13adf3f377c8f...|            2|5c818ca21204caf8c...|54965bbe3e4f07ae0...|2017-11-23 20:31:40| 160.0|        21.08|
|8c2b13adf3f377c8f...|            3|b75ad41bddb7dc94c...|1dfe5347016252a78...|2017-11-23 20:31:40|  61.0|        21.08|
|8c2b13adf3f377c8f...|            4|601a360bd2a916ece...|7a67c85e85bb2ce85...|2017-11-23 20:31:40|129.99|        42.16|
+--------------------+-------------+--------------------+--------------------+-------------------+------+-------------+



## Exploratory Data Analysis (EDA)

### How many orders we have for each status?

In [40]:
orders_df.groupBy("order_status") \
    .agg(F.count(F.col("order_id")).alias("total_orders")) \
    .orderBy(F.desc("total_orders")) \
    .show()

+------------+------------+
|order_status|total_orders|
+------------+------------+
|   delivered|       96478|
|     shipped|        1107|
|    canceled|         625|
| unavailable|         609|
|    invoiced|         314|
|  processing|         301|
|     created|           5|
|    approved|           2|
+------------+------------+



### What is the total number of orders per month, per week, per time of the day (Dawn, Morning, Afternoon, Night)?

Line Chart

1. Is there any growing trend on brazilian e-commerce?
2. On what day of week brazilians customers tend to do online purchasing?
3. What time brazilians customers tend do buy (Dawn, Morning, Afternoon or Night)?

#### Transform the order_purchase_timestamp to get the following:

Date Dimension:

- Year Month
- Day of Week
- Time of Day

In [41]:
orders_processed_df = orders_df.withColumn("year", F.date_format(F.col("order_purchase_timestamp"), "yyyy")) \
    .withColumn("year_month", F.date_format(F.col("order_purchase_timestamp"), "yyyyMM")) \
    .withColumn("day_of_week", F.date_format(F.col("order_purchase_timestamp"), "EEE")) \
    .withColumn(
        "time_of_day",
        F.when(
            F.date_format(F.col("order_purchase_timestamp"), "HH:mm").between("00:01", "05:00"), "Dawn"
        ).when(
            F.date_format(F.col("order_purchase_timestamp"), "HH:mm").between("05:01", "11:59"), "Morning"
        ).when(
            F.date_format(F.col("order_purchase_timestamp"), "HH:mm").between("12:00", "18:00"), "Afternoon"
        ).when(
            F.date_format(F.col("order_purchase_timestamp"), "HH:mm").between("18:01", "24:00"), "Night"
        )
    )

### Create a Comparison Between 2017 and 2018 Total Orders (Month by Month and YoY)

In [42]:
orders_processed_df.groupBy("year_month") \
    .agg(F.count("order_id").alias("total_orders")) \
    .orderBy("year_month") \
    .show()

+----------+------------+
|year_month|total_orders|
+----------+------------+
|    201609|           4|
|    201610|         324|
|    201612|           1|
|    201701|         800|
|    201702|        1780|
|    201703|        2682|
|    201704|        2404|
|    201705|        3700|
|    201706|        3245|
|    201707|        4026|
|    201708|        4331|
|    201709|        4285|
|    201710|        4631|
|    201711|        7544|
|    201712|        5673|
|    201801|        7269|
|    201802|        6728|
|    201803|        7211|
|    201804|        6939|
|    201805|        6873|
+----------+------------+
only showing top 20 rows



In [43]:
orders_processed_df.groupBy("year") \
    .agg(F.count("order_id").alias("total_orders")) \
    .orderBy("year") \
    .show()

+----+------------+
|year|total_orders|
+----+------------+
|2016|         329|
|2017|       45101|
|2018|       54011|
+----+------------+



### Geospatial Visualization

#### Data Processing Required
1. Merge the orders data to order_items data;
2. Use an API (brazilian government) to return the region of each customer_state;
3. Replace portuguese phonetic with regular english letters.
4. Purpose useful charts to answear business questions.

In [44]:
regions = {'AC': 'North', 'RO': 'North', 'AM': 'North', 'RR': 'North', 'PA': 'North', 'AP': 'North', 'TO': 'North', 'MT': 'Center-West', 'GO': 'Center-West', 'DF': 'Center-West', 'MS': 'Center-West', 'MA': 'Northeast', 'PI': 'Northeast', 'BA': 'Northeast', 'SE': 'Northeast', 'AL': 'Northeast', 'PE': 'Northeast', 'PB': 'Northeast', 'RN': 'Northeast', 'CE': 'Northeast', 'SP': 'Southeast', 'MG': 'Southeast', 'ES': 'Southeast', 'RJ': 'Southeast', 'PR': 'South', 'SC': 'South', 'RS': 'South'}

def map_region(state: str) -> str:
    return regions.get(state)

In [45]:
udf_map_region = F.udf(map_region, StringType())
udf_normalize_phonetic = F.udf(unidecode, StringType())

In [46]:
geolocation_df = geolocation_df.withColumn("region", udf_map_region(F.col("geolocation_state"))) \
    .withColumn("geolocation_city", udf_normalize_phonetic(F.col("geolocation_city")))

#### Total Number of Customer Order by Region, State, City

- Comparison of Each Region Orders per Month
- Top 10 Brazilian Cities with More Orders
- Total of Customers Orders by State

In [47]:
orders_processed_df.selectExpr("year_month", "order_id", "customer_id AS order_customer_id") \
    .join(customers_df, F.col("customer_id") == F.col("order_customer_id"), "inner") \
    .join(geolocation_df, F.col("customer_zip_code_prefix") == F.col("geolocation_zip_code_prefix"), "inner") \
    .groupBy("year_month", "region") \
    .agg(F.count("order_id").alias("total_orders")).show()


+----------+-----------+------------+
|year_month|     region|total_orders|
+----------+-----------+------------+
|    201707|  Northeast|       31750|
|    201705|  Southeast|      431289|
|    201709|  Northeast|       34847|
|    201802|  Northeast|       43483|
|    201808|      South|      118319|
|    201805|Center-West|       25874|
|    201701|      South|       23396|
|    201710|  Northeast|       37928|
|    201711|      South|      150286|
|    201709|      South|       87854|
|    201706|Center-West|       13256|
|    201708|  Southeast|      503106|
|    201610|Center-West|         994|
|    201801|  Northeast|       53751|
|    201702|  Southeast|      209056|
|    201709|  Southeast|      499296|
|    201609|      South|         103|
|    201807|  Southeast|      720710|
|    201803|      South|      141282|
|    201707|      South|       84224|
+----------+-----------+------------+
only showing top 20 rows



In [48]:
orders_processed_df.selectExpr("year_month", "order_id", "customer_id AS order_customer_id") \
    .join(customers_df, F.col("customer_id") == F.col("order_customer_id"), "inner") \
    .join(geolocation_df, F.col("customer_zip_code_prefix") == F.col("geolocation_zip_code_prefix"), "inner") \
    .groupBy("year_month", "geolocation_state") \
    .agg(F.count("order_id").alias("total_orders")).show()


+----------+-----------------+------------+
|year_month|geolocation_state|total_orders|
+----------+-----------------+------------+
|    201806|               SE|        2128|
|    201703|               TO|         459|
|    201807|               AP|         509|
|    201701|               AC|          45|
|    201712|               PA|        5303|
|    201708|               DF|        3832|
|    201710|               MA|        3476|
|    201808|               PB|        1341|
|    201806|               SP|      369622|
|    201805|               ES|       21592|
|    201702|               CE|         836|
|    201805|               GO|        8368|
|    201702|               PA|        1979|
|    201702|               MA|         862|
|    201708|               MA|        3237|
|    201705|               DF|        2790|
|    201707|               GO|        5647|
|    201701|               DF|         569|
|    201712|               RJ|      182098|
|    201704|               MS|  

In [49]:
orders_processed_df.selectExpr("year_month", "order_id", "customer_id AS order_customer_id") \
    .join(customers_df, F.col("customer_id") == F.col("order_customer_id"), "inner") \
    .join(geolocation_df, F.col("customer_zip_code_prefix") == F.col("geolocation_zip_code_prefix"), "inner") \
    .groupBy("year_month", "geolocation_city") \
    .agg(F.count("order_id").alias("total_orders")).show()


+----------+--------------------+------------+
|year_month|    geolocation_city|total_orders|
+----------+--------------------+------------+
|    201711|           sao paulo|       99528|
|    201710|itapecerica da serra|         308|
|    201804|      ribeirao preto|        5445|
|    201706|     franco da rocha|         376|
|    201701|              franca|         563|
|    201702|             atibaia|         324|
|    201801|santa cruz das pa...|         344|
|    201712|            itapolis|         209|
|    201705|      paulo de faria|          56|
|    201801|           brodowski|         242|
|    201801|       pariquera-acu|         148|
|    201808|     pindamonhangaba|         223|
|    201701|             vinhedo|         181|
|    201806|           itirapina|          80|
|    201707|            capivari|         321|
|    201807|       volta redonda|        1550|
|    201709|         teresopolis|         634|
|    201709|          seropedica|          43|
|    201711| 

#### How customers are distributed in Brazil? (Map and Heatmap)

In [50]:
### Dimensional Modeling

#### Total Orders vs. Total Amount Sold
- Per Month (Line)
- Per year (Number)

In [55]:
orders_processed_df.selectExpr("year_month", "year", "order_id", "customer_id AS order_customer_id") \
    .join(order_items_df, order_items_df["order_id"] == orders_df["order_id"], "inner") \
    .groupBy("year_month") \
    .agg(F.round(F.sum(order_items_df["price"]), 2).alias("total_amount_sold")) \
    .show()


+----------+-----------------+
|year_month|total_amount_sold|
+----------+-----------------+
|    201702|        247303.02|
|    201801|        950030.36|
|    201808|        854686.33|
|    201704|        359927.23|
|    201806|        865124.31|
|    201802|        844178.71|
|    201703|         374344.3|
|    201807|        895507.22|
|    201710|        664219.43|
|    201609|           267.36|
|    201709|        624401.69|
|    201805|        996517.68|
|    201711|       1010271.37|
|    201706|         433038.6|
|    201610|         49507.66|
|    201803|        983213.44|
|    201708|        573971.68|
|    201705|        506071.14|
|    201707|        498031.48|
|    201804|        996647.75|
+----------+-----------------+
only showing top 20 rows



In [56]:
orders_processed_df.selectExpr("year_month", "year", "order_id", "customer_id AS order_customer_id") \
    .join(order_items_df, order_items_df["order_id"] == orders_df["order_id"], "inner") \
    .groupBy("year") \
    .agg(F.round(F.sum(order_items_df["price"]), 2).alias("total_amount_sold")) \
    .show()


+----+-----------------+
|year|total_amount_sold|
+----+-----------------+
|2016|         49785.92|
|2017|       6155806.98|
|2018|        7386050.8|
+----+-----------------+



#### How the total sales (sum of price) are concentraded in brazilian states?

- Average price per Customer State
- Total price by Customer State

In [61]:
orders_processed_df.selectExpr("year_month", "order_id", "customer_id AS order_customer_id") \
    .join(customers_df, F.col("customer_id") == F.col("order_customer_id"), "inner") \
    .join(geolocation_df, F.col("customer_zip_code_prefix") == F.col("geolocation_zip_code_prefix"), "inner") \
    .join(order_items_df, order_items_df["order_id"] == orders_df["order_id"], "inner") \
    .groupBy("geolocation_state") \
    .agg(F.round(F.sum(order_items_df["price"]), 2).alias("total_price"), F.round(F.mean(order_items_df["price"]), 2).alias("average_price")).show()


+-----------------+--------------+-------------+
|geolocation_state|   total_price|average_price|
+-----------------+--------------+-------------+
|               SC| 7.966642329E7|       127.41|
|               RO|    3577073.58|       150.51|
|               PI|    4581195.05|       172.94|
|               AM|     825147.21|       131.67|
|               RR|     360027.85|       149.33|
|               GO| 2.086094592E7|       134.62|
|               TO|    3350329.32|       168.46|
|               MT| 2.277707282E7|       156.63|
|               SP|7.1183874052E8|       111.28|
|               ES| 4.363487856E7|       123.36|
|               PB|    6278650.25|       198.86|
|               RS|1.1118313956E8|       120.18|
|               MS|    9891112.52|        139.1|
|               AL|     7191886.1|       196.64|
|               MG|3.9719015587E8|       121.18|
|               PA| 1.558618017E7|       166.98|
|               BA| 6.237731167E7|       149.64|
|               SE| 

#### What are the best states to buy in Brazil? An analysis on sales, freight and delivery time

- Average Freight Paid for Online Shopping
- Top 5 States with Highest Avg Freight Value
- Top 5 States with Lowest Average Freight Value

- Average Delay (days) for delivery for online shopping
- Top 5 States with Highest Average Time to Delivery
- Top 5 States with Lowest Average Time to Delivery
- Average Difference between delivery and estimated date
- Top 5 States Delivery is Really Fast
- Top 5 States Delivery is not so Fast


In [62]:
order_items_df.agg(F.mean("freight_value").alias("average_freight_paid")).show()

+--------------------+
|average_freight_paid|
+--------------------+
|  19.990319928982977|
+--------------------+



In [67]:
order_items_df.join(orders_df, orders_df["order_id"] == order_items_df["order_id"], "inner") \
    .join(customers_df, customers_df["customer_id"] == orders_df["customer_id"], "inner") \
    .join(geolocation_df, F.col("customer_zip_code_prefix") == F.col("geolocation_zip_code_prefix"), "inner") \
    .groupBy("geolocation_state") \
    .agg(F.mean("freight_value").alias("average_freight_paid")) \
    .orderBy(F.desc("average_freight_paid")).show(n=5)

+-----------------+--------------------+
|geolocation_state|average_freight_paid|
+-----------------+--------------------+
|               PB|   42.77269312387491|
|               RR|   42.46960182496962|
|               PI|   39.47732502831053|
|               AC|   39.09837253960492|
|               MA|   38.07533863274666|
+-----------------+--------------------+
only showing top 5 rows



In [68]:
order_items_df.join(orders_df, orders_df["order_id"] == order_items_df["order_id"], "inner") \
    .join(customers_df, customers_df["customer_id"] == orders_df["customer_id"], "inner") \
    .join(geolocation_df, F.col("customer_zip_code_prefix") == F.col("geolocation_zip_code_prefix"), "inner") \
    .groupBy("geolocation_state") \
    .agg(F.mean("freight_value").alias("average_freight_paid")) \
    .orderBy("average_freight_paid").show(n=5)

+-----------------+--------------------+
|geolocation_state|average_freight_paid|
+-----------------+--------------------+
|               SP|  15.409965070013303|
|               PR|  20.147980714975045|
|               MG|   20.45899544966129|
|               RJ|  20.898423604479333|
|               DF|   21.01097098246382|
+-----------------+--------------------+
only showing top 5 rows



In [82]:
orders_df.withColumn("time_to_deliver", F.datediff(F.col("order_delivered_customer_date"), F.col("order_purchase_timestamp"))) \
    .agg(F.mean("time_to_deliver").alias("avg_time_to_deliver")) \
    .show()

+-------------------+
|avg_time_to_deliver|
+-------------------+
| 12.497336125046644|
+-------------------+



In [85]:
orders_df.withColumn("time_to_deliver", F.datediff(F.col("order_delivered_customer_date"), F.col("order_purchase_timestamp"))) \
    .join(customers_df, customers_df["customer_id"] == orders_df["customer_id"], "inner") \
    .join(geolocation_df, geolocation_df["geolocation_zip_code_prefix"] == customers_df["customer_zip_code_prefix"], "inner") \
    .groupBy("geolocation_state") \
    .agg(F.round(F.mean("time_to_deliver"), 2).alias("avg_time_to_deliver")) \
    .orderBy("avg_time_to_deliver") \
    .show(n=5)

+-----------------+-------------------+
|geolocation_state|avg_time_to_deliver|
+-----------------+-------------------+
|               SP|               8.87|
|               PR|              11.44|
|               MG|              11.82|
|               DF|              12.89|
|               RJ|               14.9|
+-----------------+-------------------+
only showing top 5 rows



In [83]:
orders_df.withColumn("time_to_deliver", F.datediff(F.col("order_delivered_customer_date"), F.col("order_purchase_timestamp"))) \
    .join(customers_df, customers_df["customer_id"] == orders_df["customer_id"], "inner") \
    .join(geolocation_df, geolocation_df["geolocation_zip_code_prefix"] == customers_df["customer_zip_code_prefix"], "inner") \
    .groupBy("geolocation_state") \
    .agg(F.round(F.mean("time_to_deliver"), 2).alias("avg_time_to_deliver")) \
    .orderBy(F.desc("avg_time_to_deliver")) \
    .show(n=5)

+-----------------+-------------------+
|geolocation_state|avg_time_to_deliver|
+-----------------+-------------------+
|               AP|              28.42|
|               AM|              25.03|
|               RR|              24.92|
|               AL|              23.52|
|               PA|              22.95|
+-----------------+-------------------+
only showing top 5 rows



In [75]:
orders_df.withColumn("delivery_delay", F.datediff(F.col("order_delivered_customer_date"), F.col("order_estimated_delivery_date"))) \
    .where(F.expr("delivery_delay > 0")) \
    .agg(F.mean("delivery_delay").alias("avg_days_delay")) \
    .show()

+----------------+
|  avg_days_delay|
+----------------+
|10.6203519510329|
+----------------+



In [79]:
orders_df.withColumn("delivery_delay", F.datediff(F.col("order_delivered_customer_date"), F.col("order_estimated_delivery_date"))) \
    .join(customers_df, customers_df["customer_id"] == orders_df["customer_id"], "inner") \
    .join(geolocation_df, geolocation_df["geolocation_zip_code_prefix"] == customers_df["customer_zip_code_prefix"], "inner") \
    .where(F.expr("delivery_delay > 0")) \
    .groupBy("geolocation_state") \
    .agg(F.round(F.mean("delivery_delay"), 2).alias("avg_days_delay")) \
    .orderBy(F.desc("avg_days_delay")) \
    .show(n=5)

+-----------------+--------------+
|geolocation_state|avg_days_delay|
+-----------------+--------------+
|               AP|        106.55|
|               AM|         36.61|
|               SE|         19.58|
|               RN|         18.83|
|               AC|         16.63|
+-----------------+--------------+
only showing top 5 rows



In [80]:
orders_df.withColumn("delivery_delay", F.datediff(F.col("order_delivered_customer_date"), F.col("order_estimated_delivery_date"))) \
    .join(customers_df, customers_df["customer_id"] == orders_df["customer_id"], "inner") \
    .join(geolocation_df, geolocation_df["geolocation_zip_code_prefix"] == customers_df["customer_zip_code_prefix"], "inner") \
    .where(F.expr("delivery_delay > 0")) \
    .groupBy("geolocation_state") \
    .agg(F.round(F.mean("delivery_delay"), 2).alias("avg_days_delay")) \
    .orderBy("avg_days_delay") \
    .show(n=5)

+-----------------+--------------+
|geolocation_state|avg_days_delay|
+-----------------+--------------+
|               RO|          6.21|
|               TO|          6.37|
|               DF|          6.87|
|               PR|          7.26|
|               SP|          8.81|
+-----------------+--------------+
only showing top 5 rows



## Payment Type Analysis

- Total Transactions by Payment Type
- Distribution of Payment Installments
- Total Orders Purcharsed By Month

In [89]:
order_payments_df.groupBy("payment_type") \
    .agg(F.count("order_id").alias("total_orders")) \
    .orderBy(F.desc("total_orders")).show()

+------------+------------+
|payment_type|total_orders|
+------------+------------+
| credit_card|       76795|
|      boleto|       19784|
|     voucher|        5775|
|  debit_card|        1529|
| not_defined|           3|
+------------+------------+



In [88]:
order_payments_df.groupBy("payment_installments") \
    .agg(F.count("order_id").alias("total_orders")) \
    .orderBy("payment_installments").show()

+--------------------+------------+
|payment_installments|total_orders|
+--------------------+------------+
|                   0|           2|
|                   1|       52546|
|                   2|       12413|
|                   3|       10461|
|                   4|        7098|
|                   5|        5239|
|                   6|        3920|
|                   7|        1626|
|                   8|        4268|
|                   9|         644|
|                  10|        5328|
|                  11|          23|
|                  12|         133|
|                  13|          16|
|                  14|          15|
|                  15|          74|
|                  16|           5|
|                  17|           8|
|                  18|          27|
|                  20|          17|
+--------------------+------------+
only showing top 20 rows



In [92]:
orders_processed_df.join(order_payments_df, order_payments_df["order_id"] == orders_df["order_id"], "inner") \
    .groupBy("year_month") \
    .agg(F.round(F.sum("payment_value"), 2).alias("total_payment")) \
    .orderBy(F.desc("year_month")).show()

+----------+-------------+
|year_month|total_payment|
+----------+-------------+
|    201810|       589.67|
|    201809|      4439.54|
|    201808|   1022425.32|
|    201807|   1066540.75|
|    201806|    1023880.5|
|    201805|   1153982.15|
|    201804|   1160785.48|
|    201803|   1159652.12|
|    201802|    992463.34|
|    201801|   1115004.18|
|    201712|    878401.48|
|    201711|    1194882.8|
|    201710|    779677.88|
|    201709|    727762.45|
|    201708|    674396.32|
|    201707|    592382.92|
|    201706|    511276.38|
|    201705|    592918.82|
|    201704|    417788.03|
|    201703|     449863.6|
+----------+-------------+
only showing top 20 rows

